In [1]:
'''
Checking Vector Jacobian Products For a simple neural net
'''

'\nChecking Vector Jacobian Products For a simple neural net\n'

In [2]:
import torch
import torch.nn.functional as F
from torch.autograd.functional import vjp

In [3]:
# Later increase dimensions to include batches

#FORWARD PASS

x = torch.randn(3,1,dtype=torch.float)

In [4]:
W1 = torch.randn(4,3,dtype= torch.float)
b1 = torch.randn(4,1,dtype= torch.float)

In [5]:
print(W1.mm(x)+b1)

tensor([[ 1.0209],
        [-2.6919],
        [-2.0171],
        [ 0.6815]])


In [6]:
z = W1.mm(x)+b1

In [7]:
h = F.relu(z)

In [8]:
W2 = torch.randn(2,4,dtype= torch.float)
b2 = torch.randn(2,1,dtype= torch.float)
y = W2.mm(h) + b2
y.squeeze(-1)

tensor([-0.7647,  1.8457])

In [9]:
t = torch.ones(2)

In [10]:
L = F.mse_loss(t,y)

/home/akashe/anaconda3/envs/latest_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  """Entry point for launching an IPython kernel.


In [11]:
print(L)

tensor(1.9146)


In [12]:
L_bar = 1

In [13]:
y_bar = vjp(func=F.mse_loss,inputs=(y,t))[1][0]
print(y_bar)

tensor([[-1.7647],
        [ 0.8457]])


/home/akashe/anaconda3/envs/latest_pytorch/lib/python3.6/site-packages/torch/autograd/functional.py:251: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  outputs = func(*inputs)


In [14]:
print(vjp(func=F.mse_loss,inputs=(y,t)))

(tensor(1.9146), (tensor([[-1.7647],
        [ 0.8457]]), tensor([0.4595, 0.4595])))


In [15]:
'''
Actually here derivative of t is wrong. It shud just be negative of y derivatives. It comes out right in case 
of batch products
'''

'\nActually here derivative of t is wrong. It shud just be negative of y derivatives. It comes out right in case \nof batch products\n'

In [16]:
def affine(W,h,b):
    return W.mm(h)+ b
print(vjp(func=affine,inputs=(W2,h,b2),v = y_bar)[0])
print(vjp(func=affine,inputs=(W2,h,b2),v = y_bar)[1])

tensor([[-0.7647],
        [ 1.8457]])
(tensor([[-1.8015, -0.0000, -0.0000, -1.2026],
        [ 0.8634,  0.0000,  0.0000,  0.5764]]), tensor([[4.5251],
        [4.0231],
        [0.2786],
        [2.2719]]), tensor([[-1.7647],
        [ 0.8457]]))


In [17]:
print(h)

tensor([[1.0209],
        [0.0000],
        [0.0000],
        [0.6815]])


In [18]:
h_bar = vjp(func=affine,inputs=(W2,h,b2),v = y_bar)[1][1]
z_bar = vjp(func= F.relu, inputs= z,v = h_bar)
print(z_bar)

(tensor([[1.0209],
        [0.0000],
        [0.0000],
        [0.6815]]), tensor([[4.5251],
        [0.0000],
        [0.0000],
        [2.2719]]))


In [19]:
'''
Dimensions and values of backward gradients correct. Only problem is ouput of func is calculated twice once in
forward pass and second in vjp. I will also have to take care of not updating variables untill both forward and 
backward passes are done.

Now trying a smaller network with batch size
'''

'\nDimensions and values of backward gradients correct. Only problem is ouput of func is calculated twice once in\nforward pass and second in vjp. I will also have to take care of not updating variables untill both forward and \nbackward passes are done.\n\nNow trying a smaller network with batch size\n'

In [20]:
batch_size = 10

x_ = torch.randn([batch_size,3,10],dtype=torch.float)
W_ = torch.randn([10,3],dtype= torch.float)
b_ = torch.randn([10,3,3], dtype = torch.float)

#torch.mm() works only for 2D matrices
#torch.matmul does batchwise multiplication if second tensor's dim > 2

In [21]:
z_ = x_.matmul(W_) + b_ 
print(z)

tensor([[ 1.0209],
        [-2.6919],
        [-2.0171],
        [ 0.6815]])


In [22]:
y_ = F.relu(z_)
print(y_)

tensor([[[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.3719, 3.5614, 0.0000]],

        [[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 5.2051],
         [0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.3510, 7.5721],
         [0.5425, 3.8101, 0.0000],
         [3.2170, 4.5491, 0.0000]],

        [[5.4406, 0.0671, 1.7786],
         [0.0000, 0.0000, 2.6822],
         [1.7786, 0.0000, 2.8287]],

        [[0.0000, 0.0000, 0.0000],
         [2.3336, 0.3413, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[1.1211, 0.0000, 4.3627],
         [0.6245, 0.0000, 3.5186],
         [2.4213, 0.0000, 0.0000]],

        [[3.9692, 1.4611, 0.8233],
         [3.6343, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.0000, 1.0428, 0.1825],
         [0.9822, 5.5571, 7.5583],
         [0.3741, 0.0000, 1.4823]],

        [[1.7012, 0.0000, 3.3715],
         [0.6248, 0.0000, 9.2199],
         [0.4672, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 6.4885],
  

In [23]:
t_ = torch.ones([10,3,3])

In [24]:
L_ = F.mse_loss(y_,t_)

In [25]:
print(vjp(func = F.mse_loss,inputs = (y_,t_)))
#y_bar_ = vjp(func = F.mse_loss,inputs = (y_,t_))[1][0]
#print(y_bar_)
#print(y_bar_.size())

(tensor(4.3259), (tensor([[[-0.0222, -0.0222, -0.0222],
         [-0.0222, -0.0222, -0.0222],
         [-0.0140,  0.0569, -0.0222]],

        [[-0.0222, -0.0222, -0.0222],
         [-0.0222, -0.0222,  0.0934],
         [-0.0222, -0.0222, -0.0222]],

        [[-0.0222, -0.0144,  0.1460],
         [-0.0102,  0.0624, -0.0222],
         [ 0.0493,  0.0789, -0.0222]],

        [[ 0.0987, -0.0207,  0.0173],
         [-0.0222, -0.0222,  0.0374],
         [ 0.0173, -0.0222,  0.0406]],

        [[-0.0222, -0.0222, -0.0222],
         [ 0.0296, -0.0146, -0.0222],
         [-0.0222, -0.0222, -0.0222]],

        [[ 0.0027, -0.0222,  0.0747],
         [-0.0083, -0.0222,  0.0560],
         [ 0.0316, -0.0222, -0.0222]],

        [[ 0.0660,  0.0102, -0.0039],
         [ 0.0585, -0.0222, -0.0222],
         [-0.0222, -0.0222, -0.0222]],

        [[-0.0222,  0.0010, -0.0182],
         [-0.0004,  0.1013,  0.1457],
         [-0.0139, -0.0222,  0.0107]],

        [[ 0.0156, -0.0222,  0.0527],
         [-0.008

In [26]:
# trying with 7 final output units

_x = torch.randn([10,3],dtype= torch.float)
_W = torch.randn([3,7],dtype= torch.float)
_b = torch.randn([10,7], dtype = torch.float)

_z = _x.matmul(_W) + _b
_y = F.relu(_z)
_t = torch.randn([10,7], dtype= torch.float)

_y_bar = vjp(F.mse_loss,(_y,_t))
print(_y_bar)

#gradients are correct

(tensor(2.5331), (tensor([[-0.0288,  0.0748, -0.0235,  0.0174,  0.0370,  0.0179,  0.0535],
        [-0.0209,  0.0503,  0.0575, -0.0254,  0.0442,  0.0181,  0.1107],
        [ 0.0062,  0.0140,  0.0040, -0.0078,  0.0539, -0.0180,  0.0410],
        [ 0.0026,  0.0352, -0.0010,  0.0587,  0.0151, -0.0267, -0.0062],
        [ 0.0227, -0.0108,  0.0868, -0.0336,  0.0287,  0.0240,  0.0232],
        [ 0.0447, -0.0520, -0.0099,  0.0314, -0.0373,  0.0443,  0.0257],
        [ 0.0605,  0.1180,  0.0655,  0.0210,  0.0165,  0.1052,  0.0367],
        [-0.0136,  0.0263,  0.1437,  0.0613, -0.0089, -0.0297,  0.0452],
        [ 0.0203, -0.0213, -0.0101,  0.0088, -0.0206,  0.0607,  0.0984],
        [-0.0157, -0.0199,  0.0145,  0.0085, -0.0081,  0.0183,  0.0083]]), tensor([[ 0.0288, -0.0748,  0.0235, -0.0174, -0.0370, -0.0179, -0.0535],
        [ 0.0209, -0.0503, -0.0575,  0.0254, -0.0442, -0.0181, -0.1107],
        [-0.0062, -0.0140, -0.0040,  0.0078, -0.0539,  0.0180, -0.0410],
        [-0.0026, -0.0352,  0.0

In [27]:
_x = torch.randn([2,3],dtype= torch.float)
_W = torch.randn([3,1],dtype= torch.float)
_b = torch.randn([2,1], dtype = torch.float)

_z = _x.matmul(_W) + _b
_y = F.relu(_z)
_t = torch.randn(2, dtype= torch.float) #Fail
#_t = torch.randn([2,1], dtype= torch.float) #Pass . With exact shape it passes

_y_bar = vjp(F.mse_loss,(_y,_t))
print(_y_bar)


(tensor(1.0808), (tensor([[-0.7681],
        [-0.7681]]), tensor([1.4687, 0.0675])))


In [37]:
'''
Alright I figured out the problem. when the shape of target variables is exactly same as to input then the gradients
come out right. but lets say if the shape of target is [2] and input is [2,1] then gradients arent correct because
there is another operation involved in the middle for reshaping.

I WILL HAVE TO BE VERY CAREFUL WITH SHAPES COZ broadcasting CHANGES GRADIENT when used for calculating loss in 
MSE as mentioned in the user warning..so, TARGET SHUD ALWAYS BE OF SAME DIMENSION 
ALSO, VJP ONLY TAKES TENSORS AS INPUT

Actually lets check the gradients using backward()
'''

#x = torch.rand(2)
#print(vjp(torch.reshape,(x,(-1*torch.ones(1)).data)))

_x = torch.randn([2,3],dtype= torch.float,requires_grad = True)
_W = torch.randn([3,1],dtype= torch.float, requires_grad = True)
_b = torch.randn([2,1], dtype = torch.float, requires_grad = True)

_z = _x.matmul(_W) + _b
_y = F.relu(_z)
_y.retain_grad()
_t = torch.randn(2,requires_grad=True) # Pass only with exact shape [2,1]


_loss = F.mse_loss(_y,_t)
_loss.backward()

print(_t)
print(_y)

print(_t.grad)
print(_y.grad)


tensor([-0.1756,  0.4214], requires_grad=True)
tensor([[0.0640],
        [0.1851]], grad_fn=<ReluBackward0>)
tensor([-0.3001,  0.2968])
tensor([[-0.0589],
        [ 0.0622]])


/home/akashe/anaconda3/envs/latest_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


In [44]:
_x = torch.randn([2,3],dtype= torch.float)
_W = torch.randn([3,1],dtype= torch.float)
_b = torch.randn([2,1], dtype = torch.float)

_z = _x.matmul(_W) + _b
_y = F.relu(_z)
_t = torch.randn([2,1], dtype= torch.float)
_r = torch.randn([2,1], dtype= torch.float)

def loss(y,t,r):
    return torch.sum(torch.mul((y-t),(y-t)))/len(y)

_y_bar = vjp(func=loss,inputs=(_y,_t,_r))
print(_y_bar)

(tensor(2.4084), (tensor([[-0.1140],
        [ 2.1917]]), tensor([[ 0.1140],
        [-2.1917]]), tensor([[0.],
        [0.]])))
